In [ ]:
import os
import cassio
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"

groq_api_key = os.getenv("GROQ_API_KEY")
astra_db_id = os.getenv("ASTRA_DB_ID")
astra_db_application_token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")

cassio.init(
    token=astra_db_application_token,
    database_id=astra_db_id
)

In [2]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

web_documents=loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
docs = text_splitter.split_documents(web_documents)

In [4]:
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Cassandra

embeddings = OllamaEmbeddings(model="nomic-embed-text")
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="ragverse",
    session=None,
    keyspace=None
)

In [5]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

astra_vector_store.add_documents(docs)

astra_vector_index = VectorStoreIndexWrapper(
    vectorstore=astra_vector_store
)

In [30]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=groq_api_key,
    temperature=0.7
)
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")


In [31]:
astra_vector_index.query("question='Give an overview of the agent system.'}",llm=llm)

"A LLM-powered autonomous agent system consists of a large language model (LLM) as its core controller, complemented by several key components. These components include:\n\n1. **Planning**: The agent breaks down large tasks into smaller, manageable subgoals, and can reflect on past actions to improve future results.\n2. **Memory**: The agent has short-term and long-term memory capabilities, allowing it to learn from experience and retain information over time.\n3. **Tool use**: The agent can learn to call external APIs to access extra information, execute code, and access proprietary information sources.\n\nThe LLM functions as the agent's brain, enabling it to behave conditioned on past experience and interact with other agents. The system combines these components to enable the agent to solve complex tasks, learn from mistakes, and improve its performance over time."

In [32]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

retriever = astra_vector_store.as_retriever()
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
retrieval_chain = create_retrieval_chain(
    retriever,
    document_chain
)

In [33]:
response = retrieval_chain.invoke({"input": "Give an overview of the agent system."})

In [34]:
response['answer']

"Based on the provided context, the agent system overview can be broken down into several key components, with the Large Language Model (LLM) functioning as the agent's brain. The main components are:\n\n1. **Planning**: This involves two sub-components:\n   - **Subgoal and decomposition**: The agent breaks down large tasks into smaller, manageable subgoals. This enables the efficient handling of complex tasks by dividing them into simpler, more manageable parts.\n   - **Reflection and refinement**: The agent can perform self-criticism and self-reflection over past actions. This allows the agent to learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n2. **Memory**: Although the context provided does not delve deeply into the specifics of the memory component, it is mentioned as a crucial part of the agent system, suggesting that the agent's ability to recall past experiences and learned information is vital for its operation and decis